## Simple Square Root

*TODO: Compare to bisection method*

Simple algorithm for the square root of $k$, where $k>0$:

$x_{t+1}=\frac{1}{2}(x_t+\frac{k}{x_t})$

Derivation:

$x^2=k$

$x=\frac{k}{x}$

$\frac{x}{2}=\frac{k}{2x}=\frac{1}{2}(\frac{k}{x})$

$\frac{x}{2}+\frac{x}{2}=\frac{1}{2}(\frac{k}{x})+\frac{x}{2}$

$x=\frac{1}{2}(x+\frac{k}{x})$

In [ ]:
import math

def sqrt(k: float, x0: float, t: int) -> float:
    """
    Estimate the square root of k with t iterations
    :param k: a positive number
    :param x0: initial guess of the square root of k
    :param t: number of iterations
    :return: the estimated square root of k
    """
    if t == 0:
        return x0
    prev = sqrt(k, x0, t - 1)
    return 0.5*(prev + (k / prev))

print("Estimate:", sqrt(7, 1, 5))
print("Exact:", math.sqrt(7))

How fast do different values of $x_0$ converge to the square root?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from typing import List, Dict

def sqrt_iterations(k: float, prev: float, target: float, tolerance: float) -> List[float]:
    """
    Estimate the square root of k until within tolerance
    :param k: a positive number
    :param prev: initial estimate
    :param target: actual square root
    :param tolerance: desired distance from target
    :return: a list of estimates from each iteration
    """
    new_value = 0.5*(prev + (k / prev))
    if abs(new_value - target) <= tolerance:
        return [new_value]
    return [new_value] + sqrt_iterations(k, new_value, target, tolerance)

plt.figure("sqrt")
plt.plot([1.] + sqrt_iterations(7, 1, math.sqrt(7), 0.00001))
plt.plot([5.] + sqrt_iterations(7, 5, math.sqrt(7), 0.00001))
plt.plot([50.] + sqrt_iterations(7, 50, math.sqrt(7), 0.00001))
plt.xlabel("Iteration")
plt.ylabel("Distance from target")
plt.show()

## Newton-Raphson Method

A centuries-old method for finding the zeros of a nonlinear univariate function.

For a function $f(x)$, at any point $x_t$, we can approximate the value of $f(x_{t+1})$ where $\Delta x=x_{t+1}-x_t$:

$f(x_{t+1})=f(x_t+\Delta x)\approx f(x_t)+f^{'}(x_t)\Delta x$

To get the value at the next point, take the current value and add the slope multiplied by the distance to the next point.

Solve for $x_{t+1}$:

$x_{t+1}\approx x_t+\frac{f(x_{t+1})-f(x_t)}{f^{'}(x_t)}$

To find the root, we're always hoping that $f(x_{t+1})$ will be 0. So we can approximate:

$x_{t+1}=x_t-\frac{f(x_t)}{f^{'}(x_t)}$

Consider the equation $x^x=e^x, x\in[0, \infty)$. There are two roots, $0$ and $e$.

To find them we can use $f(x)=x^x-e^x=0$.

*TODO: Animate the algorithm*

In [ ]:
%matplotlib inline
import sympy as sp

def newton(equation: sp.Expr, derivative: sp.Expr, prev_x: float, tolerance: float) -> List[float]:
    """
    Attempt to find a root of equation within tolerance
    :param equation: a single variable equation
    :param derivative: the derivative of equation
    :param prev_x: initial estimate
    :param tolerance: desired distance from 0
    :return: a list of estimates from each iteration
    """
    variables = list(equation.free_symbols)
    if len(variables) != 1:
        raise ValueError('Must be a single variable equation')
    variable = variables[0]
    new_x = prev_x - (equation.subs(variable, prev_x).evalf() / derivative.subs(variable, prev_x).evalf())
    new_y = equation.subs(variable, new_x).evalf()
    if not new_y.is_real:
        return []
    if new_y <= tolerance:
        return [new_x]
    return [new_x] + newton(equation, derivative, new_x, tolerance)

x = sp.symbols('x')
eq = x**x - sp.exp(x)

plt.figure("newton1")
for i in range(10):
    plt.plot([i] + newton(eq, sp.diff(eq), i, 0.00001))
plt.xlabel("Iteration")
plt.ylabel("Distance from target")
plt.show()

We can extend the method to find the maximum or minimum of a function $f(\textbf{x})$, by finding the critical points or roots of $f'(x)=0$ in a $d$-dimensional space.

$x^{t+1}=x^t-\frac{f'(x^t)}{f''(x^t)}=A(x^t)$

We can also extend to work with multivariate functions by replacing $x$ with the vector $\textbf{x}$, where
$\textbf{x}=(x_1, x_2, ..., x_d)^T$ is a vector of $d$ variables, and the superscript $T$ means transpose, converting the row vector to a column.

*TODO: Why do we need to transpose?*

The minimum/maximum method may converge slowly as $f'(x)\rightarrow0$. Can be improved:

$x^{t+1}=x^t-p\frac{f'(x^t)}{f''(x^t)}=A(x^t, p)$

$p=\frac{1}{1-A'(x_*)}$, where $x_*$ is the optimal solution.

*TODO: Not sure how to use this $p$ formula. How do we estimate $p$?*

For a multivariate $f(\textbf{x})$, consider the Taylor expansion around a known point $\textbf{x}=\textbf{x}_t$, and $\Delta\textbf{x}=\textbf{x}-\textbf{x}_t$.

See https://www.youtube.com/watch?v=3d6DsjIBzJ4 to refresh your memory on Taylor series.

$f(\textbf{x})=f(\textbf{x}_t)+(\nabla f(\textbf{x}_t))^T\Delta \textbf{x}+\frac{1}{2}\Delta \textbf{x}^T\nabla ^2f(\textbf{x}_t)\Delta \textbf{x}+...$

Here $\nabla$ is the vector differential operator, denoting the gradient of the function.

We can minimize $f(\textbf{x})$ by solving for $\Delta x$ in $\nabla f(\textbf{x}_t)+\nabla ^2f(\textbf{x}_t)\Delta\textbf{x}=0$.

*TODO: Why?*

Let's solve for $\textbf{x}$:

$\nabla f(\textbf{x}_t)+\nabla ^2f(\textbf{x}_t)\Delta\textbf{x}=0$

$\nabla f(\textbf{x}_t)+\nabla ^2f(\textbf{x}_t)(\textbf{x}-\textbf{x}_t)=0$

$\nabla f(\textbf{x}_t)+\nabla ^2f(\textbf{x}_t)\textbf{x}-\nabla ^2f(\textbf{x}_t)\textbf{x}_t=0$

$\nabla f(\textbf{x}_t)+\nabla ^2f(\textbf{x}_t)\textbf{x}=\nabla ^2f(\textbf{x}_t)\textbf{x}_t$

$\frac{\nabla f(\textbf{x}_t)}{\nabla ^2f(\textbf{x}_t)}+\textbf{x}=\textbf{x}_t$

$\textbf{x}=\textbf{x}_t-\frac{\nabla f(\textbf{x}_t)}{\nabla ^2f(\textbf{x}_t)}$

$\textbf{x}=\textbf{x}_t-H^{-1}\nabla f(\textbf{x}_t)$

$H$ is the [Hessian matrix](https://en.wikipedia.org/wiki/Hessian_matrix):

$\mathbf H(\textbf{x})\equiv \nabla ^2f(\textbf{x})\equiv \begin{bmatrix}
   \dfrac{\partial^2 f}{\partial x_1^2} & \dfrac{\partial^2 f}{\partial x_1\,\partial x_2} & \cdots & \dfrac{\partial^2 f}{\partial x_1\,\partial x_n} \\[2.2ex]
   \dfrac{\partial^2 f}{\partial x_2\,\partial x_1} & \dfrac{\partial^2 f}{\partial x_2^2} & \cdots & \dfrac{\partial^2 f}{\partial x_2\,\partial x_n} \\[2.2ex]
   \vdots & \vdots & \ddots & \vdots \\[2.2ex]
   \dfrac{\partial^2 f}{\partial x_n\,\partial x_1} & \dfrac{\partial^2 f}{\partial x_n\,\partial x_2} & \cdots & \dfrac{\partial^2 f}{\partial x_n^2}
 \end{bmatrix}$

Restating the min/max formula above in these terms:

$\textbf{x}^{(t+1)}=\textbf{x}^{(t)}-H^{-1}(\textbf{x}^{(t)})\nabla f(\textbf{x}^{(t)})$

This is inefficient for nonquadratic functions. *TODO: Why? Try it*

To speed it up, we use a smaller step size $\alpha\in(0, 1]$.

$\textbf{x}^{(t+1)}=\textbf{x}^{(t)}-\alpha H^{-1}(\textbf{x}^{(t)})\nabla f(\textbf{x}^{(t)})$

We can speed up further by using the identity matrix $I$ instead of calculating $H$. *TODO: Try with the Hessian too*

$\textbf{x}^{(t+1)}=\textbf{x}^{(t)}-\alpha I\nabla f(\textbf{x}^{(t)})$

This is the **steepest descent method**: taking small steps down the steepest slope from the current point to find the lowest value of the objective function.

Repeating our derivation of the Newton-Raphson method for multivariate functions:

$\Delta s=\textbf{x}^{(t+1)}-\textbf{x}^{(t)}$

$f(\textbf{x}^{(t+1)})=f(\textbf{x}^{(t)}+\Delta s)\approx f(\textbf{x}^{(t)})+(\nabla f(\textbf{x}^{(t)}))^T\Delta s$

We are trying to descend to the lowest value of $f(\textbf{x})$, so we want $(\nabla f(\textbf{x}^{(t)}))^T\Delta s$ to be negative.

The inner product $u^Tv$ of two vectors $u$ and $v$ is most negative when they are parallel in opposite directions. *TODO: Demonstrate this*

So $\Delta s$ must be parallel opposite to $\nabla f(\textbf{x}^{(t)})$: $\Delta s=-\alpha \nabla f(\textbf{x}^{(t)})$

$\alpha$ is again the step size between $0$ and $1$. We want it to be large enough to make progress without overshooting.
We can change it at each iteration for best efficiency by minimizing $f(\textbf{x}^{(t)}-\alpha ^{(t)}\nabla f(\textbf{x}^{(t)}))$, which will be a single variable optimization problem.

Our final formula is now:

$f(\textbf{x}^{(t+1)})=f(\textbf{x}^{(t)})-\alpha^{(t)}(\nabla f(\textbf{x}^{(t)}))^T\nabla f(\textbf{x}^{(t)})$

*TODO: Is alpha being called as a function here? Fix above to match what we do in code*

At each step we'll first optimize $\alpha$, using the value and gradient of $f(\textbf{x}^{(t)})$, then solve for $\textbf{x}^{(t+1)}$.

Let's try to minimize the following:

$f(x_1,x_2)=10x_1^2+5x_1x_2+10(x_2-3)^2$

$(x_1,x_2)\in [-10, 10]\times [-15, 15]$

Start at a corner: $\textbf{x}^{(0)}=(10,15)^T$

In [ ]:
# TODO: visualizations
def newton_minimum_multivariate(
        function: sp.Expr,
        initial_value: Dict[sp.Symbol, float],
        tolerance: float,
        estimates: List[Dict[sp.Symbol, float]] = None,
        max_iterations: int = 100
    ) -> Dict[sp.Symbol, float]:
    """
    Attempt to find a minimum of a multivariate function with the Newton-Raphson method
    :param function: a multivariate function
    :param initial_value: initial values of the free variables in function
    :param tolerance: how close estimates should be to count as having converged
    :param estimates: a list to populate with estimates from each iteration
    :param max_iterations: how many iterations to try before giving up
    :return: the estimated minimum, or None if the algorithm failed to converge within max_iterations
    """
    variables = list(function.free_symbols)
    if len(variables) != len(initial_value):
        raise ValueError(f'Found {len(variables)} variables, but initial_value of length {len(initial_value)}')
    if estimates is not None:
        estimates.append(initial_value)
    gradient = {v: sp.diff(f, v) for v in variables}
    alpha = sp.symbols('alpha')
    def go(prev_value: Dict[sp.Symbol, float], iterations: int):
        if iterations >= max_iterations:
            return None
        gradient_prev = {v: gradient[v].evalf(subs=prev_value) for v in variables}
        alpha_params = {v: prev_value[v] - alpha*gradient_prev[v] for v in variables}
        alpha_optimize = function.subs(alpha_params)
        # TODO: this should work, but alpha_value is converging to a value and then the actual minimization never converges. Investigate
        #alpha_value = newton(alpha_optimize, alpha_optimize.diff(), 0.5, tolerance)[-1]
        alpha_value = sp.solve(alpha_optimize.diff(), alpha)[0]
        new_value = {v: prev_value[v] - (alpha_value * gradient_prev[v]) for v in variables}
        if estimates is not None :
            estimates.append(new_value)
        if abs(sum([new_value[v] - prev_value[v] for v in variables]) / len(variables)) <= tolerance:
            return new_value
        return go(new_value, iterations + 1)
    return go(initial_value, 0)

x1, x2 = sp.symbols('x1 x2')
f = 10*x1**2 + 5*x1*x2 + 10*(x2 - 3)**2
initial = {x1: 10, x2: 15}

f_estimates = []
print(newton_minimum_multivariate(f, initial, 0.0001, f_estimates))
print(f_estimates)

## Formalizing Optimization Problems

We have a **design vector** $\textbf{x}=(x_1, x_2, ..., x_D)^T$.
Components $x_i$ of the $D$-dimensional vector $\textbf{x}$ are **design** (or decision) **variables**.
They span the **design** (or search) **space** $\mathbb{R}^D$.

We need to minimize some number of **objective** (or cost) **functions** $f_i(\textbf{x}), (i=1, 2, ..., M), \textbf{x}\in\mathbb{R}^D$.
That is, $M$ functions of the design vector. These form the **solution** (or response) **space**.

We are subject to some **constraints**: $h_j(\textbf{x})=0, (j=1, 2, ..., J)$ and $g_k(\textbf{x})\leq0, (k=1, 2, ..., K)$.

It is technically possible to have only constraints and no objectives. This is a **feasibility problem**, where any solution is optimal.

Characteristics that make problems more difficult:

- **Nonlinearity**, objective functions that don't map linearly from design vector to solution space
- **Multimodality**, multiple local optima
- **High dimensionality**, the size of the design space increases faster than the space an algorithm can search
- **Constraints**, which can split feasible regions into complicated and disconnected shapes

*TODO: Use Shekel and Ackley functions as worst-case tests. Also: https://www.sfu.ca/~ssurjano/optimization.html*

## Hill Climbing with Random Restart

When searching for the maximum of a function, there may be multiple global maxima, or misleading local maxima.
Many algorithms will find the maximum closest to the provided starting point, meaning the solution found will be somewhat random.

To avoid this and make sure we search the whole design space, we can restart the algorithm many times from different randomly selected starting points.

Gradient-based algorithms will still run into problems even with random restart, when dealing with complex real-world problems that don't have derivatives.
It's also difficult to know how many random trials to do without knowing how many local peaks exist in advance.

> Therefore, gradient-free methods are preferred. In fact, modern nature-inspired algorithms are almost all gradient-free optimization methods.

## No-Free-Lunch Theorems

> If any algorithm A outperforms another algorithm B in the search for an extremum of an objective function, then algorithm B will outperform A over other objective functions.

> All algorithms for optimization will give the same average performance when averaged over *all possible* functions, which means that the universally best method does not exist for all optimization problems.

> For any specific set of objective functions, some algorithms can perform much better than others.
